# Aula 9 - InPars

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)


# Enunciado exercício



Objetivo: gerar dataset para treino de modelos de buscas usando a técnica do InPars e avaliar um modelo reranqueador treinado neste dataset no TREC-COVID:

Entrada: 3-5 exemplos few-shot + documento amostrado da coleção do TREC-COVID

Saída: query que seja relevante para o documento amostrado

É opcional fazer a etapa de filtragem usando as queries de maior prob descrita no Artigo.

Como modelo gerador, use um dos seguintes modelos:
ChatGPT-3.5-turbo: ~1 USD para cada 1k exemplos
FLAN-T5 (base, large ou XL), LLAMA-(7,13B), Alpaca-(7/13B), que são possiveis de rodar no Colab Pro.

Também tem a inference-api da HF: https://huggingface.co/inference-api.

Com exceção do LLAMA, é possivel usar zero-shot ao inves de few-shot.
Dado 1k-10k pares <query sintética; documento>, treinar um modelo reranqueador miniLM igual ao da aula 2/3.

Exemplos negativos (i.e., <query sintética; doc não relevant) vem do BM25: dado a query sintetica, retornar top 1000 com o BM25, e amostrar aleatoriamente alguns documentos como negativo

Começar treino do miniLM já treinado no MS MARCO

Avaliar no TREC-COVID e comparar com o reranqueador apenas treinado no MSMARCO

Nota: Também usar o dataset dos colegas para obter diversidade de exemplos: Assim que tiver gerado o dataset sintético, favor colocar na planilha, assim outras pessoas podem usa-lo.
- Para aumentar a aleatoriedade, seed usada deve o seu numero na planilha.

Colocar dataset no formato jsonlines:
{"query": query, "positive_doc_id": doc_id, "negative_doc_ids": [opcional]}\n 



Dicas: (do exercício da aula 2)

- Siga sempre um padrão ao criar os exemplos few-shot. Aqui tem uma pagina com dicas para prompt engineering: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


- Usar a API do LLAMA fornecida por nós (licença exclusiva para pesquisa). [Colab demo da API do LLAMA](https://colab.research.google.com/drive/1zZ-ch29LTicNPA62t2MaOwMROywnqUxf?usp=sharing) (obrigado, Thales Rogério)
- Opcionalmente, usar a API do code-davinci-002, que é de graça e trás resultados muito bons.
CUIDADO: NÃO USAR O TEXT-DAVINCI-002/003, que é pago

- Opcionalmente, usar a API do ChatGPT (gpt-3.5-turbo) que é barata: ~1 centavo de real por 1000 tokens (uma página)
  
- Opcionalmente, usar o Alpaca: https://alpaca-ai.ngrok.io/



Este caderno contempla os passos 3 (filtrar por relevância) e 4 (encontrar exemplos negativos) do [fluxo de processamento](https://github.com/marcusborela/deep_learning_em_buscas_unicamp/blob/main/presentations/articles/Aula%208%20-%20InPars%20Process.png)

# Organizando o ambiente

## Importações

In [363]:
import requests  # para Llama
import time # para Llama

In [364]:
from tqdm import tqdm

In [365]:
import string

In [366]:
import gzip

In [367]:
import getpass

In [368]:
import json, time

In [369]:
from transformers import BatchEncoding

In [370]:
from torch.utils.data import DataLoader

In [371]:
from torch.utils.data import Dataset

In [372]:
from psutil import virtual_memory

In [373]:
import pandas as pd
import os

In [374]:
import random
import numpy as np
import torch

In [375]:
import transformers

In [376]:
# para limpar texto
import ftfy

In [377]:
import pickle

## Definindo paths

In [378]:
DIRETORIO_LOCAL = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local'
DIRETORIO_TRABALHO = F'{DIRETORIO_LOCAL}/inpars'
DIRETORIO_TREC_COVID = F'{DIRETORIO_LOCAL}/trec_covid'

In [379]:
DIRETORIO_TREC_COVID_CARDIN = F'{DIRETORIO_TRABALHO}/cardin'
CAMINHO_CARDIN_TREC_COVID = f"{DIRETORIO_TREC_COVID_CARDIN}/trec-covid-inpars.jsonl"

In [380]:
DIRETORIO_INDICE = f"{DIRETORIO_TRABALHO}/indexes/trec-covid-index"

In [381]:
if os.path.exists(DIRETORIO_LOCAL):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_LOCAL)
    print('pasta criada!')


pasta já existia!


In [382]:
if os.path.exists(DIRETORIO_TREC_COVID_CARDIN):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_TREC_COVID_CARDIN)
    print('pasta criada!')


pasta já existia!


In [383]:
if os.path.exists(DIRETORIO_TRABALHO):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_TRABALHO)
    print('pasta criada!')


pasta já existia!


## Função de verificação de memória

In [384]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue May  2 22:14:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 66

In [385]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [386]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 49.13
 used 17.03
 free 28.91
 cached 20.95
 buffers 0.46
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue May  2 22:14:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+==================

### Vinculando pasta do google drive para salvar dados

## Fixando as seeds

In [387]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


A semente dever ser meu número na planilha da tarefa no [classoom](https://docs.google.com/spreadsheets/u/0/d/1mvA8ZrN2FjPxIDwJkYJGsVdZNH49Z1w1L0_3pIAZhQo/htmlview#gid=0)

In [388]:
num_semente = 13 

In [389]:
inicializa_seed(num_semente)

## Preparando para debug e display

In [390]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [391]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [392]:
config_display()

In [393]:
config_debug()

# Carregando queries geradas e textos dos documentos

## Carregando as queries

In [394]:
with open(f"{DIRETORIO_TRABALHO}/queries_geradas_102878.pickle", 'rb') as outputFile:
    dict_queries_geradas = pickle.load(outputFile)

In [395]:
len(dict_queries_geradas)

102878

## Baixando o dataset trec-covid para buscar o texto e depois gerar índice para busca dos exemplos negativos

In [396]:
if not os.path.exists(f"{DIRETORIO_TREC_COVID}/corpus.jsonl.gz"):
    !wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
    !mv corpus.jsonl.gz {DIRETORIO_TREC_COVID}
    print('Baixado')
else:
    print('Já existia a pasta')

Já existia a pasta


In [397]:
# Descompacte o arquivo para a memória
with gzip.open(f'{DIRETORIO_TREC_COVID}/corpus.jsonl.gz', 'rt') as f:
    # Leia o conteúdo do arquivo descompactado
    corpus = [json.loads(line) for line in f]

In [398]:
# Exiba os dados carregados
print(f"{type(corpus)} len(corpus): {len(corpus)} corpus[0] {corpus[0]}" )

<class 'list'> len(corpus): 171332 corpus[0] {'_id': 'ug7v899j', 'title': 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia', 'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%

In [399]:
corpus_dict = {}

for docto in corpus:
    if ('title' in docto) and len(docto['title']) >= 5:
        texto_usado_na_geracao_de_query = docto['title'] + '. ' + docto['text']
    else:
        texto_usado_na_geracao_de_query = docto['text']
    corpus_dict[docto['_id']] = {'text_query_generation': texto_usado_na_geracao_de_query, 
                                 'title': docto['title'],
                                 'text': docto['text']}

In [400]:
print(len(corpus_dict))

171332


In [401]:
corpus_dict['ug7v899j'].keys(),corpus_dict['ug7v899j']['text_query_generation']

(dict_keys(['text_query_generation', 'title', 'text']),
 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty

# Filtro de queries mais relevantes

Lógica:

         Para cada par docto com query gerada (<= usa: dict_queries_geradas):
            Calcular índice de relevância conforme rankeador 
         Ordenar pelos primeiros <qtd_documento_filtrado> documentos 


## Calcular relevância entre queries geradas e documentos

In [403]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [404]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [405]:
num_workers_dataloader = 0

In [406]:
#const_nome_modelo = 'microsoft/MiniLM-L12-H384-uncased'
const_nome_modelo = 'cross-encoder/ms-marco-TinyBERT-L-2'
model = AutoModelForSequenceClassification.from_pretrained(const_nome_modelo).to(device)
tokenizer = AutoTokenizer.from_pretrained(const_nome_modelo)

In [407]:
model.config.max_position_embeddings

512

In [408]:
tokenizer.max_len_sentences_pair

509

In [409]:
tokenizer.model_max_length

512

### Criando código para o Dataset

In [410]:
class MyDataset(Dataset):
    """
      Classe para representar um dataset de texto e classes.
    """  
    def __init__(self, texts: np.ndarray, classes:list[int], tokenizer):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (np.ndarray): um array com as strings de texto. Cada linha deve ter 2 strings.
          classes (np.ndarray): um array com as classes de cada texto.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, np.ndarray), f"Parâmetro texts deve ser do tipo np.ndarray e não {type(texts)}"
      assert texts.shape[1] == 2, "Array must have 2 columns"
      for row in texts:
          assert isinstance(row, np.ndarray) and row.shape == (2,), f"Each row in texts must have 2 elements"
          assert isinstance(row[0], str) and isinstance(row[1], str), f"Each element in texts.row must be a string e não {type(row[0])}"
      assert isinstance(classes,np.ndarray), f'classes deve ser do tipo np.ndarray e não {type(classes)}'
      assert isinstance(classes[0],np.int64), f'classes[0] deve ser do tipo numpy.int64 e não {type(classes[0])} '

      self.texts = texts
      self.classes = classes
      self.tokenizer = tokenizer
      self.max_seq_length = tokenizer.model_max_length

      # Salvar os dados dos tensores
      x_data_input_ids = []
      x_data_token_type_ids = []
      x_data_attention_masks = []
      for text_pair in tqdm(texts, desc='encoding text pair'):
          encoding = tokenizer.encode_plus(
              text_pair[0],
              text_pair[1],
              add_special_tokens=True,
              max_length=self.max_seq_length,
              padding='max_length',
              return_tensors = 'pt',
              truncation=True,
              return_attention_mask=True,
              return_token_type_ids=True
          )
          x_data_input_ids.append(encoding['input_ids'].long())
          x_data_token_type_ids.append(encoding['token_type_ids'].long())
          x_data_attention_masks.append(encoding['attention_mask'].long())
      print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      # squeeze: vai transformar um tensor de shape [2, 1, 322] em um tensor de shape [2, 322].

      self.x_tensor_input_ids = torch.stack(x_data_input_ids).squeeze(1)
      self.x_tensor_attention_masks = torch.stack(x_data_attention_masks).squeeze(1)
      self.x_tensor_token_type_ids = torch.stack(x_data_token_type_ids).squeeze(1)

    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.texts)
    
    def __getitem__(self, idx):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode 
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        return {
            'input_ids': self.x_tensor_input_ids[idx],
            'attention_mask': self.x_tensor_attention_masks[idx],
            'token_type_ids': self.x_tensor_token_type_ids[idx],
            # 'labels': int(self.classes[idx])
            'labels': torch.tensor(self.classes[idx], dtype=torch.long)            
        }

#### Testando o MyDataset e o Dataloader

In [411]:
# Cria dados fictícios
texts = np.array([['This is the first query', 'text associated with query 1'],
                  ['This is the second query', 'text associated with query 2'],
                  ['This is the third query', 'text associated with query 3'],
                  ['This is the fourth query', 'text associated with query 4'],
                  ['This is the fifty query', 'text associated with query 5'],
                  ['This is the sixty query', 'text associated with query 6'],])
classes = np.ones(len(texts), dtype=int)


In [412]:
# Cria um objeto da classe MyDataset
dummy_dataset = MyDataset(texts=texts, classes=classes, tokenizer=tokenizer)

encoding text pair: 100%|██████████| 6/6 [00:00<00:00, 2786.30it/s]

	Vou converter lista para tensor;  Momento: [2023-mai-02 22:15:34]


In [413]:
# Testa o método __len__()
assert len(dummy_dataset) == 6

# Testa o método __getitem__()
sample = dummy_dataset[0]
assert set(sample.keys()) == {'input_ids', 'attention_mask', 'token_type_ids', 'labels'} # 
assert isinstance(sample['input_ids'], torch.Tensor)
assert sample['input_ids'].shape[0] == tokenizer.model_max_length
assert isinstance(sample['attention_mask'], torch.Tensor)
assert sample['attention_mask'].shape[0] == tokenizer.model_max_length
# assert isinstance(sample['labels'], int)
assert isinstance(sample['labels'], torch.Tensor)
print(sample['labels'].shape, sample['labels'], sample['input_ids'].shape)
print(sample['input_ids'][:20])


torch.Size([]) tensor(1) torch.Size([512])
tensor([  101,  2023,  2003,  1996,  2034, 23032,   102,  3793,  3378,  2007,
        23032,  1015,   102,     0,     0,     0,     0,     0,     0,     0])


In [414]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers= num_workers_dataloader)

In [415]:
first_batch = next(iter(dummy_loader))

In [416]:
first_batch = BatchEncoding(first_batch).to(device)

In [417]:
first_batch

{'input_ids': tensor([[ 101, 2023, 2003,  ...,    0,    0,    0],
        [ 101, 2023, 2003,  ...,    0,    0,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0')}

Confirmando efeito do parâmetro shuffle em DataLoader

In [418]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=num_workers_dataloader)
for ndx, epoch in tqdm(enumerate(range(1)), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")


Epochs: 1it [00:00, 868.39it/s]

tensor([1, 1])
tensor([1, 1])
tensor([1, 1])
Fim época 0


In [419]:
del dummy_loader, dummy_dataset

### Calculando relevância para os pares

In [420]:
# Criar um DataFrame a partir do dicionário
df = pd.DataFrame.from_dict(dict_queries_geradas, orient='index', columns=['generated_query'])
df['id_docto'] = df.index


In [421]:
df = df.reset_index(drop=True)

In [422]:
df.head()

,generated_query,id_docto
0,What is the significance of the finding that i...,ug7v899j
1,What is the significance of the presumed contr...,02tnwd4m
2,What is the significance of SP-D in the innate...,ejv2xln0
3,What is the significance of ET-1 in lung disease?,2b73a28n
4,What is the significance of the mucosal respon...,9785vg6d


In [423]:
df.shape

(102878, 2)

In [424]:
df['text_docto_query_generation'] = df['id_docto'].apply(lambda x: corpus_dict[x]['text_query_generation'])


In [425]:
df.head()

,generated_query,id_docto,text_docto_query_generation
0,What is the significance of the finding that i...,ug7v899j,Clinical features of culture-proven Mycoplasma...
1,What is the significance of the presumed contr...,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...
2,What is the significance of SP-D in the innate...,ejv2xln0,Surfactant protein-D and pulmonary host defens...
3,What is the significance of ET-1 in lung disease?,2b73a28n,Role of endothelin-1 in lung disease. Endothel...
4,What is the significance of the mucosal respon...,9785vg6d,Gene expression in epithelial cells in respons...


In [426]:
def calcula_relevancia(parm_model, parm_dataloader_reranking):
  # para 'cross-encoder/ms-marco-TinyBERT-L-2'
  prob_relevancia = []
  parm_model.eval()
  with torch.no_grad():
      for ndx, batch in tqdm(enumerate(parm_dataloader_reranking), total=len(parm_dataloader_reranking), mininterval=0.5, desc='dataset_reranking', disable=False):
          logits_model = parm_model(**BatchEncoding(batch).to(device)).logits                          
          relevantes_float = [float(t) for t in logits_model]
          prob_relevancia.extend(relevantes_float)          
          # prob_relevancia.append(pa.array(scores.numpy()))
  return prob_relevancia

In [427]:
%%time
print("carregando dataset")  
classes_dummy = np.zeros(len(dict_queries_geradas), dtype=np.int64)
print(classes_dummy.shape)
dataset_reranking = MyDataset(texts=df[['generated_query','text_docto_query_generation']].values, classes=classes_dummy, tokenizer=tokenizer)    
# torch.save(dataset_reranking, path_treino+'dataset_reranking_input_layout_qp_len_'+str(hparam['num_sentenca_valid'])+'.pt')

carregando dataset
(102878,)


encoding text pair: 100%|██████████| 102878/102878 [01:45<00:00, 979.13it/s]


	Vou converter lista para tensor;  Momento: [2023-mai-02 22:17:43]
CPU times: user 1min 45s, sys: 688 ms, total: 1min 46s
Wall time: 1min 46s


In [428]:
# Limpa o cache da memória da GPU
torch.cuda.empty_cache()

In [429]:
gc.collect()

0

In [431]:
batch_size = 32

In [432]:
dataloader_reranking = DataLoader(dataset_reranking,
                                  batch_size= batch_size, 
                                  shuffle=False)

In [433]:
%%time
prob_relevancia_query = calcula_relevancia(model,dataloader_reranking)

dataset_reranking: 100%|██████████| 3215/3215 [00:23<00:00, 136.37it/s]

CPU times: user 23.5 s, sys: 21.7 ms, total: 23.5 s
Wall time: 23.6 s


In [434]:
prob_relevancia_query[:8]

[-2.144087314605713,
 3.8457999229431152,
 1.7638808488845825,
 2.4114737510681152,
 3.2398197650909424,
 3.3087615966796875,
 4.076657295227051,
 3.372711181640625]

In [435]:
df['score_relevance'] = prob_relevancia_query

In [436]:
df.head()

,generated_query,id_docto,text_docto_query_generation,score_relevance
0,What is the significance of the finding that i...,ug7v899j,Clinical features of culture-proven Mycoplasma...,-2.1440873
1,What is the significance of the presumed contr...,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,3.8457999
2,What is the significance of SP-D in the innate...,ejv2xln0,Surfactant protein-D and pulmonary host defens...,1.7638808
3,What is the significance of ET-1 in lung disease?,2b73a28n,Role of endothelin-1 in lung disease. Endothel...,2.4114738
4,What is the significance of the mucosal respon...,9785vg6d,Gene expression in epithelial cells in respons...,3.2398198


# Encontrar exemplos negativos

## Gerar índice com texto sem pré-processamento
   
      .. entrada: 171mil textos 
      .. saída: índice lucene (só de textos que tiveram queries geradas: exclui textos vazios)


### Construindo o cardin com os textos usados na geração das queries

In [264]:
DIRETORIO_TREC_COVID_CARDIN, CAMINHO_CARDIN_TREC_COVID, DIRETORIO_INDICE

('/home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/cardin',
 '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/cardin/trec-covid-inpars.jsonl',
 '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/indexes/trec-covid-index')

In [184]:
jsonl_trecc_selecionada = []
# Para cada índice único, armazena a lista de tokens correspondente
for docto in tqdm(corpus):
    if len(docto['text']) > 20:
        if ('title' in docto) and len(docto['title']) >= 5 :
            texto = docto['title'] + '. ' + docto['text'] 
        else:
            texto = docto['text'] 
        jsonl_trecc_selecionada.append(
            {"id": docto['_id'],
            "contents": texto})

100%|██████████| 171332/171332 [00:00<00:00, 675781.39it/s]


In [265]:
len(jsonl_trecc_selecionada)

128924

In [195]:
%%time
with open(CAMINHO_CARDIN_TREC_COVID, 'w') as arquivo:
    for doc in jsonl_trecc_selecionada:
        # Converte o dicionário em uma string JSON
        linha_json = json.dumps(doc)
        # Escreve a linha JSON no arquivo
        arquivo.write(f'{linha_json}\n')

CPU times: user 746 ms, sys: 108 ms, total: 854 ms
Wall time: 856 ms


### Criando o índice

In [197]:
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [198]:
from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher

In [203]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {DIRETORIO_TREC_COVID_CARDIN} \
  --index {DIRETORIO_INDICE} \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-05-01 13:30:59,395 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-05-01 13:30:59,396 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-05-01 13:30:59,396 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-05-01 13:30:59,396 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/cardin
2023-05-01 13:30:59,396 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-05-01 13:30

### Realizando a busca e atualizando em df_sorted_by_relevance a lista de exemplos negativos

In [437]:
# realizando busca e salvando dados do estágio 1
searcher = LuceneSearcher(DIRETORIO_INDICE)
searcher.set_bm25(k1=0.82, b=0.68)  


In [438]:
num_max_hits = 1000
num_negative_example = 5

In [439]:
%%time
negative_ids = []
for idx, registro in tqdm(df.iterrows()):
    query_text = registro['generated_query']
    hits = searcher.search(query_text, num_max_hits)
    if len(hits)>0:
        indices_selecao = np.random.randint(0, high=len(hits), size=num_negative_example)
        negative_ids.append([hits[ndx].docid for ndx in indices_selecao])
    else:
        negative_ids.append([])


102878it [58:46, 29.17it/s]

CPU times: user 1h 2s, sys: 3.09 s, total: 1h 5s
Wall time: 58min 46s


In [440]:
# Adicionar a lista como uma nova coluna no dataframe
df['negative_ids'] = negative_ids

Excluir queries que não lograram sucesso na busca (visto um caso que o texto era "nneneneneeeeeee")

In [441]:
df[df['negative_ids'].apply(lambda x: len(x) == 0)]

,generated_query,id_docto,text_docto_query_generation,score_relevance,negative_ids
26,𝑛𝑒𝑡𝑒𝑡𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒�,5dk231qs,Torsional restraint: a new twist on frameshift...,-6.5541410,[]
106,𝑛𝑒𝑡𝑒𝑡𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒�,noscodew,Cell-penetrating peptides as transporters for ...,-5.5144677,[]
114,𝑛𝑡𝑟𝑜𝑙𝑒𝑟𝑜𝑙𝑒𝑟𝑜𝑙𝑒𝑟𝑜𝑙𝑒𝑟𝑜𝑙𝑒𝑟𝑜𝑙𝑒𝑟�,6lezilfv,Host Gene Expression Profiling of Dengue Virus...,-6.2530360,[]
172,𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛𝑒𝑛�,fpsyem7x,Endothelial Cells Support Persistent Gammaherp...,-5.8792858,[]
206,𝑛𝑒𝑡𝑒𝑡𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒,928pkpv2,Rift Valley Fever Virus NSs Protein Promotes P...,-6.0086517,[]
...,...,...,...,...,...
102379,𝑛𝑒𝑛𝑒𝑛𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒�,fxtzitpk,Structures of MERS-CoV spike glycoprotein in c...,-6.2074203,[]
102404,?,9g3gvkgr,Numbers with little meaning. Estimates of covi...,-6.6821342,[]
102460,̶W̶h̶e̶r̶e̶ ̶i̶s̶ ̶t̶h̶e̶ ̶i̶n̶t̶e̶r̶f̶e̶n̶s̶i...,4elisewv,Antiviral Innate Immunity: Introduction☆. Abst...,-6.4218655,[]
102678,𝑛𝑡𝑟𝑜𝑙𝑒𝑟𝑜𝑛𝑒𝑟𝑜𝑙𝑒𝑟𝑜𝑛𝑒𝑟𝑜𝑙𝑒𝑟𝑜𝑛𝑒𝑟�,78dznsqw,The aqueous extract from Toona sinensis leaves...,-6.0982895,[]


In [442]:
df = df[df['negative_ids'].apply(lambda x: len(x) > 0)]

In [443]:
df.head()

,generated_query,id_docto,text_docto_query_generation,score_relevance,negative_ids
0,What is the significance of the finding that i...,ug7v899j,Clinical features of culture-proven Mycoplasma...,-2.1440873,"[onqy3xyu, 0zas25r9, 7ambsarg, d2tfg3af, 9sxxa..."
1,What is the significance of the presumed contr...,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,3.8457999,"[1a1ryj8b, ls2nud43, blbaqbo7, xn6oomgn, 4fcyo..."
2,What is the significance of SP-D in the innate...,ejv2xln0,Surfactant protein-D and pulmonary host defens...,1.7638808,"[1ijxfc1v, vtkhwj57, jqqaffaw, 3w6hivv8, y8ry0..."
3,What is the significance of ET-1 in lung disease?,2b73a28n,Role of endothelin-1 in lung disease. Endothel...,2.4114738,"[ciz3xai2, add4i1c9, ch8dfex0, u61mcemm, z8j6w..."
4,What is the significance of the mucosal respon...,9785vg6d,Gene expression in epithelial cells in respons...,3.2398198,"[l5ok3bvx, bkr3wmku, xanewi59, 2j4pjjwk, sp212..."


In [444]:
df.shape

(101878, 5)

## Ordenar pelos primeiros <qtd_documento_filtrado> documentos

In [445]:
df_sorted_by_relevance = df.sort_values('score_relevance', ascending=False)


In [446]:
df_sorted_by_relevance

,generated_query,id_docto,text_docto_query_generation,score_relevance,negative_ids
83756,What is the importance of COVID-19 information...,vpowumff,Availability of COVID-19 information from nati...,5.0976543,"[t4wofvv4, 4567mu8n, v9oqlouj, 6d25aq3i, rgeel..."
51386,What is the impact of COVID-19 lockdown on hou...,l813q689,COVID-19 Lockdown in a Kenyan Informal Settlem...,5.0703812,"[3tjem9ny, 6x0nzxpz, ev8ap5fa, udhvfos0, ge70t..."
83755,What is the importance of COVID-19 information...,pq0jrvi6,Availability of COVID-19 Information From Nati...,5.0661101,"[nfklfcw6, 4567mu8n, p3egbdrw, d0v3d730, 1i0hz..."
69750,What is the importance of risk perception asse...,nkz53jzy,Risk Perception of COVID-19 Among the Portugue...,5.0235000,"[qcr1fl8s, unzbduqk, bnnl700a, 4dv6954b, j8h7e..."
53173,"""We've Cared for the Dead since We Started Car...",g2beiwe4,“We've Cared for the Dead since We Started Car...,5.0054770,"[wlrcaypa, ceo6gfgc, 0prx2rfp, vhvngxzm, w1d2k..."
...,...,...,...,...,...
15926,何内容?,irqyqa9b,Overview of Data Visualization. This chapter w...,-7.0421448,"[f1airzpg, 9svv192k, 9svv192k, h81q345m, lze5g..."
14383,何が良いのか?,yjcb1stx,Spectral Clustering by Subspace Randomization ...,-7.0431447,"[trjtmyef, aezp1isw, k8o0cuby, 20suvitt, yk3fp..."
91591,What is the importance of early recognition an...,wkrl4hrz,A Trip through Southeast Asian Airports in Tim...,-7.0432777,"[0m3svdmy, z0sfqbfr, clc0bggd, wat7s3cl, 4k4qk..."
10131,何者が言いたいのか?,od0fr8l0,Epidemic-Logistics Network Considering Time Wi...,-7.0434837,"[04zs475e, kvbh5poh, fmnij1ta, 0kxail2v, e9nz2..."


In [447]:
df_sorted_by_relevance.shape

(101878, 5)

# Salvar arquivos

Completo

In [448]:
with open(f"{DIRETORIO_TRABALHO}/df_sorted_by_relevance_{len(df_sorted_by_relevance)}.pickle", 'wb') as outputFile:
    pickle.dump(df_sorted_by_relevance, outputFile, pickle.HIGHEST_PROTOCOL)

Apenas com os <qtd_mais_relevantes>

In [449]:
qtd_documento_filtrado = 30000

In [450]:
qtd_mais_relevantes = min(qtd_documento_filtrado,len(df_sorted_by_relevance))
qtd_mais_relevantes

30000

In [451]:
with open(f"{DIRETORIO_TRABALHO}/df_sorted_by_relevance_{qtd_mais_relevantes}_mais_relevantes.pickle", 'wb') as outputFile:
    pickle.dump(df_sorted_by_relevance[:qtd_mais_relevantes], outputFile, pickle.HIGHEST_PROTOCOL)

## Salvar dados em formato para fazer upload do split em dataset huggingface

Do meu split (usando semente meu número na planilha da tarefa no [classoom](https://docs.google.com/spreadsheets/u/0/d/1mvA8ZrN2FjPxIDwJkYJGsVdZNH49Z1w1L0_3pIAZhQo/htmlview#gid=0))

In [452]:
num_semente = 13 

In [453]:
inicializa_seed(num_semente)

In [454]:
indices_docto_my_split = np.random.randint(0, high=len(corpus)-1, size=20000)

In [455]:
list_doc_id_my_split = [corpus[ndx]['_id'] for ndx in indices_docto_my_split]

In [456]:
df_my_split = df_sorted_by_relevance[df_sorted_by_relevance['id_docto'].isin(list_doc_id_my_split)]


In [457]:
df_my_split.shape

(11322, 5)

In [458]:
num_registros = min(len(df_my_split), 1000)
num_registros

1000

In [459]:
df_my_split = df_my_split[:num_registros]

In [460]:
with open(f"{DIRETORIO_TRABALHO}/df_sorted_by_relevance_split_borela_{num_registros}_v2.pickle", 'wb') as outputFile:
    pickle.dump(df_my_split, outputFile, pickle.HIGHEST_PROTOCOL)

In [461]:
import datasets

In [300]:
datasets_username = 'unicamp-dl/trec-covid-experiment'

In [43]:
hugging_face_token = getpass.getpass('Informe seu_token de API para acesso ao hugging face')


In [357]:

ds = datasets.load_dataset(datasets_username, 
                            download_mode='force_redownload',
                            use_auth_token=hugging_face_token)
                            # Outros possíveis parâmetros
                            # split='train', 
                            # ignore_verifications=False,
                            # beam_pipeline_options=None,
                            # cache_dir=None,
                            # revision=None,


Dataset trec-covid-experiment downloaded and prepared to /home/borela/.cache/huggingface/datasets/unicamp-dl___trec-covid-experiment/default/0.0.0/39177de766cb7ace6cb0b5a27f1db36700b181fd775fbd271589004af3109267. Subsequent calls will reuse this data.


100%|██████████| 11/11 [00:00<00:00, 1420.40it/s]


In [358]:
ds

DatasetDict({
    example: Dataset({
        features: ['query', 'positive_doc_id', 'negative_doc_ids'],
        num_rows: 3
    })
    example2: Dataset({
        features: ['query', 'positive_doc_id', 'negative_doc_ids'],
        num_rows: 3
    })
    eduseiti_100_queries_expansion_20230501_01: Dataset({
        features: ['query', 'positive_doc_id', 'negative_doc_ids'],
        num_rows: 463
    })
    leandro_carisio_01: Dataset({
        features: ['query', 'positive_doc_id', 'negative_doc_ids'],
        num_rows: 1001
    })
    thales_1k_generated_queries_20230429: Dataset({
        features: ['query', 'positive_doc_id', 'negative_doc_ids'],
        num_rows: 1000
    })
    manoel_1k_generated_queries_20230430: Dataset({
        features: ['query', 'positive_doc_id', 'negative_doc_ids'],
        num_rows: 1000
    })
    manoel_2k_generated_queries_20230501: Dataset({
        features: ['query', 'positive_doc_id', 'negative_doc_ids'],
        num_rows: 2000
    })
    thiago_l

In [360]:
ds['marcus_borela_1k_gptj6b_20230501'][:1]

{'query': ['What is the significance of the speculation about the catastrophe that awaits once COVID-19 establishes itself in the poorest communities of South Africa?'],
 'positive_doc_id': ['jmn8ctlt'],
 'negative_doc_ids': [['n26csuks',
   '09jyekp5',
   'g5bhkd55',
   'x1xzcacy',
   'qv794vbt']]}

In [362]:
df = pd.concat((v.to_pandas().assign(origin=k) for k,v in ds.items()), ignore_index=True)
df


,query,positive_doc_id,negative_doc_ids,origin
0,This is a example query 1,doc1,"[xxx, yyy, zzz]",example
1,This is another example query,doc2,"[aaa, bbb, ccc]",example
2,Example of query with no negative doc_ids,doc2,[],example
3,This is a example query 1 (file 2),doc12222,"[xxx, yyy, zzz]",example2
4,This is another example query (file 2),doc12345,"[aaa, bbb, ccc]",example2
...,...,...,...,...
9443,What is the significance of the phylogeny of t...,28nv3b8a,"[0sedbv51, m8lrc4c0, i6puqauk, bp8dqis1, exlsn...",marcus_borela_1k_gptj6b_20230501
9444,What is the significance of the accuracy of th...,sffuejo1,"[go4d8jwn, n5gk1xhb, edunzo0f, xk2a7tsw, 6l888...",marcus_borela_1k_gptj6b_20230501
9445,What is the significance of the low fecundity ...,3dyatdlv,"[azv9yw9n, r55fe25x, wc23tqv2, vqmqnipq, d0dni...",marcus_borela_1k_gptj6b_20230501
9446,What is the significance of regional and natio...,jps1j60a,"[uu4k2j2a, mo4luyx6, mnt12ot2, fex8sd1t, fex8s...",marcus_borela_1k_gptj6b_20230501


In [359]:
ds['leandro_carisio_01'][:1]

{'query': ['Here are some relevant queries that the user can use to search for the given document extract:\n\n1. How many birds have been inoculated so far?\n2. What is the number of birds that officials claim to have inoculated?\n3. What is the total count of birds that have received inoculation according to officials?\n4. What is the progress of bird inoculation as claimed by officials?\n5. How successful has the bird inoculation drive been according to officials?'],
 'positive_doc_id': ['xvdlbbds'],
 'negative_doc_ids': [['sbr9qd0k',
   '850teajs',
   'm7w7aue8',
   's64v656n',
   'ea3gwnw7']]}

In [312]:
ds['mirelle_1k_generated_queries_20230501'][:2]

{'query': ['What are the clinical outcomes in patients undergoing transcatheter aortic valve replacement with percutaneous coronary intervention of the left main coronary artery, as evaluated by the TAVR-LM registry?',
  'What is the impact of the Coronavirus pandemic on crime rates in Italy and how has it been reflected in data from Ministries, Europol reports, newspapers, and news?'],
 'positive_doc_id': ['ikrs1k76', '0jv5mnnl'],
 'negative_doc_ids': [['f7s8s770',
   'f8m1b8gd',
   'mff7zlg9',
   '1bx244k1',
   'yb8r0ezg'],
  ['ijlnizev', '3mu9rrql', 'i6khpntc', 'd37t9f4i', 'h2k4utqr']]}

In [307]:
df_my_split.head()

,generated_query,id_docto,text_docto_query_generation,score_relevance,negative_ids
22994,What is the significance of the speculation ab...,jmn8ctlt,COVID-19 and tuberculosis in South Africa: A d...,4.7682772,"[n26csuks, 09jyekp5, g5bhkd55, x1xzcacy, qv794..."
13399,What is the significance of the TRAP-18 in ass...,ysz3oocp,Assessing the threat of lone-actor terrorism: ...,4.7625756,"[ayt31knx, hycd9zua, xug622im, rjzy3z8t, m54mo..."
10873,€œIn augustus 2011 wijdde de Journal of the Am...,jnuncmd5,1 Inleiding. In augustus 2011 wijdde de Journa...,4.7553005,"[tfvn7xp8, yxmqmetv, 420px62r, s3xnieig, aey40..."
22425,What is the estimated reproduction number of C...,k1qq4qgg,Epidemic curve and reproduction number of COVI...,4.7455740,"[axjrmk48, q4a1n1wm, m2bgbqzg, raav221g, oen0y..."
15497,What is the significance of automatic semantic...,c5zewz3m,Automatic Semantic Description Extraction from...,4.7357488,"[70x44y0t, 5b78ow1j, vr1kmria, 5htvzkpt, borka..."


Mudar para o formato jsonl

In [462]:
split_nome = "marcus_borela_1k_gptj6b_20230501"

In [463]:
with open(f"{DIRETORIO_TRABALHO}/{split_nome}.jsonl", 'w') as arquivo:
    for ndx, registro in tqdm(df_my_split.iterrows()):
        doc = {"query": registro['generated_query'],
                "positive_doc_id":  registro['id_docto'],
                "negative_doc_ids":  registro['negative_ids'],
                }
        json.dump(doc, arquivo)
        # linha_json = json.dumps(doc)
        # Escreve a linha JSON no arquivo
        arquivo.write('\n')        

1000it [00:00, 15136.59it/s]


In [464]:
lista_json = []
with open(f"{DIRETORIO_TRABALHO}/{split_nome}.jsonl", "r") as f:
    for line in f:
        doc = json.loads(line)
        lista_json.append(doc)

In [471]:
print(lista_json[:2])

[{'query': 'What is the importance of COVID-19 information provided by national plastic surgery society websites?', 'positive_doc_id': 'vpowumff', 'negative_doc_ids': ['t4wofvv4', '4567mu8n', 'v9oqlouj', '6d25aq3i', 'rgeeld8q']}, {'query': 'What is the importance of risk perception assessment of COVID-19 among Portuguese healthcare professionals and general population?', 'positive_doc_id': 'nkz53jzy', 'negative_doc_ids': ['qcr1fl8s', 'unzbduqk', 'bnnl700a', '4dv6954b', 'j8h7endt']}]


meu_dataset_dict = DatasetDict({
    split_nome: Dataset.from_dict(lista_json),
})

Comandos a testar depois

Adicionar o novo conjunto de dados ao ds existente
ds.update(meu_dataset_dict)


In [320]:
from datasets import Dataset, DatasetDict

In [ ]:
# carregar o novo split a partir do arquivo jsonl
new_split = datasets.load_dataset('json', data_files='marcus_borela_v1.jsonl') #['train']

# adicionar o novo split ao dataset original
ds = ds.add_dataset(new_split, name='marcus_borela_01')